# Deep Learning Model Implementation

1. Data Loading
2. Label + One-Hot Encoding
3. Tokenization + Padding
4. Embedding
5. Data Splitting
6. Model Implementation

| Model | Accuracy |
| --- | --- |
| LSTM | |
| Bi-LSTM | |
| GRU | |
| Bi-GRU | |
| 1D-CNN | |

In [1]:
import os, joblib
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Bidirectional, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense
import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier



2024-11-10 01:30:28.431418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731184228.470932   54325 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731184228.479172   54325 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 01:30:28.508137: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define Data directory
DATA_DIR = '../data/'

# list of poet names
# Removed the 5 lowest poets
poets = ["abbasinYousuf", "hamzaBaba", "karanKhan", "khatirAfridi", "khushalKhanKhattak", "matiullahTurab", "mumtazOrakazi", "munirJan", "naeemAhmed", "rabiaMumtaz", "rahmanBaba", "rehmatShah", "sahibShahSabir"]

# # Pehle k 13
# poets = ["abbasinYousuf", "azizMazerwal", "ghaniKhan", "hamzaBaba", "khaliqZiari", "khatirAfridi", "khushalKhanKhattak",  "mumtazOrakazi", "munirJan",  "rahmanBaba", "rehmatShah", "sahibShahSabir", "salimRiaz"]

## All
# poets = ["abbasinYousuf", "azizMazerwal", "ghaniKhan", "hamzaBaba", "karanKhan", "khaliqZiari", "khatirAfridi", "khushalKhanKhattak", "matiullahTurab", "mumtazOrakazi", "munirJan", "naeemAhmed", "rabiaMumtaz", "rahmanBaba", "rehmatShah", "sahibShahSabir", "salimRiaz"]

poet_labels = {poet: i for i, poet in enumerate(poets)}
print(poet_labels)

{'abbasinYousuf': 0, 'hamzaBaba': 1, 'karanKhan': 2, 'khatirAfridi': 3, 'khushalKhanKhattak': 4, 'matiullahTurab': 5, 'mumtazOrakazi': 6, 'munirJan': 7, 'naeemAhmed': 8, 'rabiaMumtaz': 9, 'rahmanBaba': 10, 'rehmatShah': 11, 'sahibShahSabir': 12}


In [3]:
# Load and Label the Data
def load_and_label(data_dir, poets, poet_labels):
    data = []
    labels = []

    for poet in poets:
        poet_dir = os.path.join(data_dir, poet)
        file_path = os.path.join(poet_dir, f'{poet}.txt')

        # Check if the file exists
        if not os.path.exists(file_path):
            print(f'{file_path} does not exist')
            continue

        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            data.extend(lines)
            labels.extend([poet_labels[poet]] * len(lines))

    df = pd.DataFrame({'text': data, 'label': labels})
    return data, labels, df

In [4]:
data, labels, df = load_and_label(DATA_DIR, poets, poet_labels)
df.describe()

,label
count,36629.000000
mean,5.326217
std,3.542465
min,0.000000
25%,3.000000
50%,4.000000
75%,8.000000
max,12.000000


In [5]:
X = df['text'].values
y = df['label'].values

# Label Encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
class PoemClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(PoemClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten along the sequence length dimension
        x = F.relu(x)
        x = self.fc(x)
        return x

In [8]:

poems_train, poems_temp, labels_train, labels_temp = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)
poems_val, poems_test, labels_val, labels_test = train_test_split(poems_temp, labels_temp, test_size=0.5, random_state=42)

tfidf_vectorizer = TfidfVectorizer(max_features=5000)

poems_train_tfidf = tfidf_vectorizer.fit_transform(poems_train).toarray()
poems_val_tfidf = tfidf_vectorizer.transform(poems_val).toarray()
poems_test_tfidf = tfidf_vectorizer.transform(poems_test).toarray()

In [9]:
poems_train_tfidf = poems_train_tfidf.reshape(poems_train_tfidf.shape[0], 1, poems_train_tfidf.shape[1])
poems_val_tfidf = poems_val_tfidf.reshape(poems_val_tfidf.shape[0], 1, poems_val_tfidf.shape[1])
poems_test_tfidf = poems_test_tfidf.reshape(poems_test_tfidf.shape[0], 1, poems_test_tfidf.shape[1])

In [10]:
class PoemClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(PoemClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = F.relu(x)
        x = self.fc(x)
        return x

In [11]:
# Plot training history
def plot_training_history(history, title="Model Training"):
    plt.figure(figsize=(12, 5))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f"{title} - Loss")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f"{title} - Accuracy")
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [12]:
# # Define the LSTM Model
# model = Sequential()
# model.add(LSTM(128, input_shape=(X_train_tfidf.shape[1], X_train_tfidf.shape[2]), return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(64, return_sequences=True))
# model.add(Dropout(0.3))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(len(set(encoded_labels)), activation='softmax'))

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Define the early stopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Train the model
# history = model.fit(X_train_tfidf, y_train, validation_data=(X_vals_tfidf, y_vals), epochs=10, batch_size=32, callbacks=[early_stopping])

model = Sequential()
model.add(Bidirectional(LSTM(30)))
model.add(Dense(len(set(encoded_labels)), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(poems_train_tfidf, labels_train, validation_data=(poems_val_tfidf, labels_val), epochs=10, batch_size=32, callbacks=[early_stopping])

W0000 00:00:1731184238.356660   54325 gpu_device.cc:2433] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
W0000 00:00:1731184238.358461   54325 gpu_device.cc:2433] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1731184238.452124   54325 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3503 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 960M, pci bus id: 0000:02:00.0, compute capability: 5.0
2024-11-10 01:30:39.481488: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 586060000 exceeds 10% of free system memory.
2024-11-10 01:30:40.239583: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 586060000 exceeds 10% of free system memor

Epoch 1/10


F0000 00:00:1731184242.605135   54325 concat_lib_gpu_impl.cu.cc:151] Non-OK-status: GpuLaunchKernel( concat_fixed_kernel<T, IntType>, config.block_count, config.thread_per_block, 0, gpu_device.stream(), input_ptrs, split_size, static_cast<int>(output->dimension(0)), static_cast<int>(output->dimension(1)), output->data())
Status: INTERNAL: no kernel image is available for execution on the device


: 

In [ ]:
# plot_training_history(history, title="Second LSTM Model (TF-IDF) Training")

# # Evaluate on the test set
# test_loss, test_accuracy = model.evaluate(X_test_tfidf, y_test)
# print(f"Test Accuracy: {test_accuracy:.4f}")

# # Make predictions on the test set
# y_probs = model.predict(X_test_tfidf)
# y_preds = np.argmax(y_probs, axis=1)

# # Confusion Matrix
# confusion_matrix = confusion_matrix(y_test, y_preds)
# plt.figure(figsize=(10, 8))
# sns.heatmap(confusion_matrix, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('Confusion Matrix - LSTM Model (TF-IDF)')

# # Classification Report
# classification_rep = classification_report(y_test, y_preds, target_names=poets)
# print(classification_rep)
# plt.show()

plot_training_history(history, title="Second LSTM Model (TF-IDF) Training")

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(poems_test_tfidf, labels_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Make predictions on the test set
y_probs = model.predict(poems_test_tfidf)
y_preds = np.argmax(y_probs, axis=1)

# Confusion Matrix
confusion_matrix = confusion_matrix(labels_test, y_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt='d', xticklabels=poets, yticklabels=poets)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - LSTM Model (TF-IDF)')
plt.show()
